In [8]:
import osmnx as ox
import networkx as nx
import datetime
from colorama import Fore
from colorama import Style

def loadingGraph(neighborhood, city, country):
    if (neighborhood):
        G = ox.graph_from_place(neighborhood + ", " + city + ", " + country, network_type='drive', simplify=True).to_undirected()
    else:
        G = ox.graph_from_place(city + ", " + country, network_type='drive', simplify=True).to_undirected()
    G = nx.convert_node_labels_to_integers(G)
    
    return G

def eulerized(G):
    H = nx.eulerize(G)
    return H

def pretty_printGPS(gps):
    print(Fore.RED + Style.BRIGHT + "Start the drone" + Style.RESET_ALL)
    i = 0
    for i in range (0 , len(gps)):
        if i == 0:
            print("Start by taking the " + Fore.GREEN + Style.DIM + gps[i] + Style.RESET_ALL)
        else:
            if gps[i] == gps[i - 1]:
                continue
            else:
                print("Next, scan " + Fore.GREEN + Style.DIM + str(gps[i]) + Style.RESET_ALL)
    print(Fore.RED + Style.BRIGHT + "Land the drone")
            

def get_GPS(H):
    tmp = nx.eulerian_circuit(H)
    list_ = []
    GPS = []
    for elem in tmp:
        list_.append(elem[0])
        name = ""
        try:
            name = H.edges[(elem[0], elem[1], 0)]['name']
            if (type(name) == list):
                for road in name:
                    GPS.append(road)
            else:
                GPS.append(name)
        except KeyError:
            continue
    return GPS,list_
    

def main(neighborhood, city, country, print_gps, print_map, print_drone):
    
    #Graph Loading
    begin_load = datetime.datetime.now()
    print(Fore.GREEN + Style.BRIGHT + "Loading graph..." + Style.RESET_ALL)
    G = loadingGraph(neighborhood,city,country)
    print(Fore.BLUE + Style.BRIGHT + "Loading graph time: " + str(datetime.datetime.now() - begin_load))
    if (print_map):
        fig, ax = ox.plot_graph(ox.project_graph(G))


    #Eulerize Graph
    begin_euler = datetime.datetime.now()
    print(Fore.GREEN + Style.BRIGHT + "Begining euleriazation...")
    H = nx.eulerize(G)
    print(Fore.BLUE + Style.BRIGHT + "Eulerize time: " + str(datetime.datetime.now() - begin_euler))
    
    #Option : print the GPS path
    liste = []
    gps,liste = get_GPS(H)
    if (print_gps):
        pretty_printGPS(gps)   
    
    #Print the figures
    if (print_drone):
        fig, ax = ox.plot_graph_route(G, liste, route_linewidth=4, node_size=1, bgcolor='k')

    print(Fore.RED + Style.BRIGHT + "Total time: " + str(datetime.datetime.now() - begin_load))

#This is where we execute the code
#First parameter represent the neighborhood if their is one
#Second paramater is mandatory and is the city
#Third parameter is the country, also mandatory
#The last three ones enable or not respectively: print the gps route, print the map, print the drone route on the map
main(None, "Gentilly", "France", True, False, False)

Loading graph...
Loading graph time: 0:00:00.864616
Begining euleriazation...
Eulerize time: 0:00:00.784959
Start the drone
Start by taking the Rue Pierre Marcel
Next, scan Avenue Paul Vaillant-Couturier
Next, scan Rue Dedouvre
Next, scan Avenue Paul Vaillant-Couturier
Next, scan Avenue Lénine
Next, scan 3e Avenue
Next, scan 2e Avenue
Next, scan 3e Avenue
Next, scan Allée du Colonel Arnaud Beltrame
Next, scan 3e Avenue
Next, scan Cité du Chaperon Vert
Next, scan 2e Avenue
Next, scan Rue Danielle Mitterrand
Next, scan Avenue du Chaperon Vert
Next, scan Avenue Lénine
Next, scan Rue Foubert
Next, scan Avenue Lénine
Next, scan Rue des Aqueducs
Next, scan Mail des Tilleuls
Next, scan Rue du Petit Bois
Next, scan Rue des Carrières
Next, scan Avenue Lénine
Next, scan Rue des Aqueducs
Next, scan Mail des Tilleuls
Next, scan Rue du Petit Bois
Next, scan Rue des Carrières
Next, scan Avenue Lénine
Next, scan Avenue Paul Vaillant-Couturier
Next, scan Rue Auguste Blanqui
Next, scan Avenue Paul Vail